In [4]:
from ortools.sat.python import cp_model
import pandas as pd
import plotly.express as px

In [5]:
### Variables ###

In [6]:
days = range(5) ###Eventually model will be extended to a month or year
#sessions = range(17)
sessions = range(19)
tracers = range(4)
scanners = range(2)

pib_mk_weight=6
mk_pib_weight=6
mk_concur_weight=3
pib_concur_weight=3
pib_weight=1
mk_weight=1
ucb_j_weight=0.25

###TODO: Add variable for staff schedules; Design matrix?
#Human Readable
day_names = ["Mon","Tues","Wed","Thurs","Fri"]
sess_names = ["08:00 - 08:30","08:30 - 09:00","09:00 - 09:30",
             "09:30 - 10:00","10:00 - 10:30","10:30 - 11:00",
             "11:00 - 11:30","11:30 - 12:00","12:00 - 12:30",
             "12:30 - 13:00","13:00 - 13:30","13:30 - 14:00",
             "14:00 - 14:30","14:30 - 15:00","15:00 - 15:30",
             "15:30 - 16:00","16:00 - 16:30","16:30 - 17:00",
              "17:00 - 17:30"]
tracer_names = ["PIB","AV1451","MK6240","UCB-J"]
scan_durations = {"PIB":3,"AV1451":3,"MK6240":2,"UCB-J":3}
tracer_isotopes = {"PIB":"11C","AV1451":"18F","MK6240":"18F","UCB-J":"11C"}
scanner_names = ["ECAT HR+","Biograph Horizon"]

In [7]:
### No use for design matrix yet but could be used to 
#weight individual sessions

design_matrix = []
#sessions
sessions_design = [1]*len(sessions)
#days
days_design = []
for day in range(len(days)):
    days_design.append(sessions_design)
#tracers
tracers_design = []
for tracer in range(len(tracers)):
    tracers_design.append(days_design)
#scanners
for scanner in range(len(scanners)):
    design_matrix.append(tracers_design)
#print(design_matrix)
#design_matrix[p][t][d][s]

In [8]:
### Creates the model CP Model: 
#https://developers.google.com/optimization/reference/python/sat/python/cp_model
model = cp_model.CpModel()

In [9]:
### Decision Variables ###

In [10]:
#Add new variable for each combination of days, sessions, tracers
#Each day,session,tracer is a Bool of whether day/session/tracer is
#instance is scheduled or not
x = {}
for p in scanners:
    for t in tracers:
        for d in days:
            for s in sessions:
                x[(t,d,s,p)] = model.NewBoolVar(
                    "{}{}{}{}".format(tracer_names[t],
                                      day_names[d],
                                      sess_names[s],
                                      scanner_names[p]))

In [11]:
### Hard Constraints ###

In [12]:
#Constraints creating distinct sessions

In [13]:
#Require each day to have at least n scans
#Push solver to schedule scans since no objective function

In [14]:
#Each session on a scanner gets at most 1 tracer
for p in scanners:
    for d in days:
        for s in sessions:
            model.Add(sum(x[(t,d,s,p)] for t in tracers) <= 1)
#Checked

In [15]:
#Prevent overlap between scans
for p in scanners:
    for d in days:
        for t1 in tracers:
            value=scan_durations[tracer_names[t1]]
            for s in sessions[0:len(sessions)-(value-1)]:
                overlap=[x[(t1,d,s,p)]]
                for blocked in range(1,value):
                    for t2 in tracers:
                        overlap.append(x[(t2,d,s+blocked,p)])
                model.Add(sum(overlap) <= 1)
#Checked

In [16]:
#Prevent scans from being scheduled in a slot where they will finish after 17:30
too_late=[]
for p in scanners:
    for d in days:
        for t in tracers:
            value=scan_durations[tracer_names[t]]
            for s in sessions[len(sessions)-(value-1):len(sessions)]:
                too_late.append(x[(t,d,s,p)])
model.Add(sum(too_late) == 0)

In [17]:
#Each tracer gets at least 1 session per week
for t in tracers:
    all_sess = []
    for p in scanners:
        for d in days:
            for s in sessions:
                all_sess.append(x[(t,d,s,p)])
    model.Add(sum(all_sess) >= 1)

In [18]:
#PIB constraints

In [19]:
#PIB 3 max per day/batch
pib = tracer_names.index("PIB")
for d in days[1:4]:
    p_day = []
    for p in scanners:
        for s in sessions:
            p_day.append(x[(pib,d,s,p)])
    model.Add(sum(p_day) <= 3)

In [20]:
#PIB 1 max per Monday
pib = tracer_names.index("PIB")
d = day_names.index("Mon")
p_mon = []
for p in scanners:
    for s in sessions:
        p_day.append(x[(pib,d,s,p)])
model.Add(sum(p_day) <= 1)

In [21]:
#PIB 3 hr gap between PIB scans
three_hours = 6 #sessions
pib = tracer_names.index("PIB")
for d in days:
    for s in sessions[0:len(sessions)-three_hours]:
        p_gap = []
        for p in scanners:
            for num in range(three_hours):
                p_gap.append(x[(pib,d,s+num,p)])
        model.Add(sum(p_gap) <= 1)

In [22]:
#AV1451 constraints

In [23]:
#AV1451 are scheduled every Friday (HR+) and every other Wednesday (Biograph)
#Put on schedule for each Wednesday and Friday - Users will remove and rerun
av = tracer_names.index("AV1451")
av_session = sessions[8]
av_week = []
wed = days[2]
fri = days[len(days)-1]
hr = scanners[0]
bio = scanners[1]

#Wed Biograph
model.Add(x[(av,wed,av_session,bio)] == 1)
#Fri HR+
model.Add(x[(av,fri,av_session,hr)] == 1)

In [24]:
#AV1451 are not scheduled by ADRC/WRAP so only have two repeating slots
av = tracer_names.index("AV1451")
a_week = []
for p in scanners:
    for d in days:
        for s in sessions:
            a_week.append(x[(av,d,s,p)])
model.Add(sum(a_week) == 2)

In [25]:
#MK6240 constraints

In [26]:
#MK6240 max 3 sessions each day
mk = tracer_names.index("MK6240")
for d in days:
    m_3 = []
    for p in scanners:
        for s in sessions:
            m_3.append(x[(mk,d,s,p)])
    model.Add(sum(m_3) <= 3)

In [27]:
#MK6240 only sessions 9 through 17; Tues-Fri
mk = tracer_names.index("MK6240")
m_days = []
#sess_not = [0,1,2,3,4,5,6,7,8,18]
for p in scanners:
    for d in days[1:5]:
        for s in list(sessions[0:9])+list(sessions[18:19]):
            m_days.append(x[(mk,d,s,p)])
model.Add(sum(m_days) == 0)

In [28]:
#MK6240 only sessions 8 through 14; Mon
mk = tracer_names.index("MK6240")
d = day_names.index("Mon")
m_mon = []
for p in scanners:
    for s in list(sessions[0:7])+list(sessions[15:19]):
        m_mon.append(x[(mk,d,s,p)])
model.Add(sum(m_mon) == 0)

In [29]:
#UCB-J constraints

In [30]:
#UCB-J max 1 session each day
ucb_j = tracer_names.index("UCB-J")
for d in days:
    u_1 = []
    for p in scanners:
        for s in sessions:
            u_1.append(x[(ucb_j,d,s,p)])
    model.Add(sum(u_1) <= 1)

In [31]:
#UCB-J 90-min gap prior to session during which other 11C tracers cannot be scanned [PIB,ER176,UCB-J]
#UCB-J constraint already included with 1 session per day constraint
#ER176 is not considered in this model
ucb_j = tracer_names.index("UCB-J")
pib = tracer_names.index("PIB")
for d in days[1:5]: #Tues-Fri
    for s_ucb_j in sessions[3:len(sessions)]:
        for p in scanners:
            #3 sessions before these sessions must be not have scans for [PIB,ER176,UCB-J]
            for blocked in range(1,4):
                u_sess = []
                u_sess.append(x[(ucb_j,d,s_ucb_j,p)])
                u_sess.append(x[(pib,d,s_ucb_j-blocked,p)])
                model.Add(sum(u_sess) <= 1)

In [32]:
### Instanciate Solver, solve, and output end states ###

In [33]:
#Class used from Google OR Docs Employee Scheduling Examples
###TODO: Rename for tracer scheduling
class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, 
                 num_shifts, scanners, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._scanners = scanners
        self._solution_count = 0
        self._solution_limit = limit
        #self.solutions = {}


    def on_solution_callback(self):      
        self._solution_count += 1
        #Create dataframe to hold results
        results = []
        #results = {}
        #results = [{"Solution": self._solution_count}]
        #print('Solution %i' % self._solution_count)
        for p in self._scanners:
            scanner = ["Scanner{}.format(p)"]
            for d in self._num_days:
                #Get each day's schedule
                day = ["Day{}".format(d)]
                #day = {}
                #day = [{"Day":d}]
                #for d in range(self._num_days):
                #print('Day %i' % d)
                for n in self._num_nurses:
                #for n in range(self._num_nurses):
                    is_working = False
                    for s in self._num_shifts:
                    #for s in range(self._num_shifts):
                        if self.Value(self._shifts[(n, d, s, p)]):
                            is_working = True
                            #print('  Nurse %i works shift %i' % (n, s))
                            day.append([d,n,s,p,1])
                            #day.update({(d,n,s) : 1})
                            #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":1})
                    if not is_working:
                        #print('  Nurse {} does not work'.format(n))
                        #day.append({"Day":d,"Tracer":n,"Session":s,"Assigned":0})
                        #day.update({(d,n,s) : 0})
                        day.append([d,n,s,p,0])
                #results.update({d : day})
                results.append(day)
            #self.solutions.update({self._solution_count : results})
        pd.DataFrame(results).to_csv("schedule_solution_{}.csv".format(self._solution_count),index=False)
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

In [34]:
#Save solutions as CSV
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
#Enumerate all solutions
solver.parameters.enumerate_all_solutions = True
#Display the first n solutions
solution_limit = 3
solution_printer = NursesPartialSolutionPrinter(x, tracers,
                                                days, sessions,
                                                scanners,
                                                solution_limit)
solver.Solve(model, solution_printer)
# Statistics.
print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())

Stop search after 3 solutions

Statistics
  - conflicts      : 0
  - branches       : 1990
  - wall time      : 0.219918 s
  - solutions found: 3


In [35]:
### Create Gantt Graph of Solution ###

In [39]:
sch_data = pd.DataFrame(columns=["Session","Start","Finish","Tracer","color"])
#seed data with a background color for each session if no scan
for entry in range(len(sess_names)):
    sch_data.loc[len(sch_data.index)] = dict(Session=sess_names[entry], Start=0, Finish=5, Tracer="", color="1")
    
#Graph solution
#Read solution from CSV and then add scheduled sessions to Gantt
sch = pd.read_csv("schedule_solution_1.csv",
                  index_col=0).T #transpose to get days as columns

#Add each scheduled scan to calendar
tracers_colors = ["2","3","4","5"]
for column in sch:
    day = sch[column]
    print(day)
    for entry in day:
        if entry == entry: #avoids NaN's created by transpose
            print(entry)
            ent_l = entry.replace(" ","").replace("[","").replace("]","").split(",")
            if int(ent_l[4]) == 1 and int(ent_l[3]) == 0: #Get first scanner schedule
                dur = scan_durations[tracer_names[int(ent_l[1])]] #get how many sessions this scan will last
                for s in range(dur):
                    sch_entry = dict(Session=sess_names[int(ent_l[2])+s], #add entry for each 30min of scan
                                    Start=int(ent_l[0]), Finish=int(ent_l[0])+1,
                                    Tracer=tracer_names[int(ent_l[1])], 
                                     color=tracers_colors[int(ent_l[1])])
                    sch_data.loc[len(sch_data.index)] = sch_entry

sch_data['delta'] = sch_data['Finish'] - sch_data['Start']
sch_data = sch_data.astype({'Tracer': 'string', 'color': 'string'})

fig_hr = px.timeline(sch_data, x_start="Start", x_end="Finish", y="Session",
                 color="color", text="Tracer")
fig_hr.update_yaxes(autorange="reversed")
fig_hr.layout.xaxis.type = 'linear'
for d in fig_hr.data:
  filt = sch_data['color'] == d.name
  d.x = sch_data[filt]['delta'].tolist()
fig_hr.show()

0              Day0              Day0
1   [0, 0, 3, 0, 1]  [0, 0, 18, 1, 0]
2  [0, 1, 18, 0, 0]  [0, 1, 18, 1, 0]
3   [0, 2, 7, 0, 1]  [0, 2, 18, 1, 0]
4   [0, 3, 0, 0, 1]  [0, 3, 18, 1, 0]
Day0


IndexError: list index out of range

In [ ]:
#fig.to_dict() #use if want to adjust formatting

In [ ]:
#Save Gantt Graph as HTML
file_name = 'scheduling_solution_1_graph'
fig.write_html(f"{file_name}.html")